In [1]:
import mmcv
from mmcv import Config
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.models import build_detector
from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
import os
from mmcv.parallel import MMDataParallel
import pandas as pd
from pandas import DataFrame
from pycocotools.coco import COCO
import numpy as np

/opt/conda/lib/python3.7/site-packages/mmcv/utils/registry.py:250: UserWarning: The old API of register_module(module, force=False) is deprecated and will be removed, please use the new API register_module(name=None, force=False, module=None) instead.
  'The old API of register_module(module, force=False) '


In [2]:
classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
# config file 들고오기
cfg = Config.fromfile('./configs/trash/faster_rcnn/vf_resnest101.py')

PREFIX = '../../input/data/'

cfg.data.samples_per_gpu = 4
cfg.seed=2020
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/resnet101_hrfpn_vfnet'
epoch = 36
checkpoint_path = os.path.join(cfg.work_dir, f'epoch_{epoch}.pth')
#checkpoint_path = os.path.join(cfg.work_dir, 'best_bbox_mAP_50.pth')

In [3]:
cfg.evaluation

{'interval': 1, 'metric': 'bbox', 'save_best': 'bbox_mAP_50'}

In [4]:
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [5]:
model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])


2021-05-19 12:42:41,493 - mmdet - INFO - load model from: torchvision://resnet101
2021-05-19 12:42:41,494 - mmdet - INFO - Use load_from_torchvision loader
2021-05-19 12:42:41,989 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias



Use load_from_local loader


In [6]:
output = single_gpu_test(model, data_loader, show_score_thr=0.05)

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 837/837, 8.1 task/s, elapsed: 104s, ETA:     0s

In [7]:
prediction_strings = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
imag_ids = coco.getImgIds()

class_num = 11
for i, out in enumerate(output):
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    for j in range(class_num):
        for o in out[j]:
            prediction_string += str(j) + ' ' + str(o[4]) + ' ' + str(o[0]) + ' ' + str(o[1]) + ' ' + str(
                o[2]) + ' ' + str(o[3]) + ' '
        
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])


submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join(cfg.work_dir, f'submission_{epoch}.csv'), index=None)
submission.head()

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


,PredictionString,image_id
0,0 0.030689314 0.0 263.06998 21.266968 303.166 ...,batch_01_vt/0021.jpg
1,0 0.043173574 283.237 232.90508 484.0651 427.2...,batch_01_vt/0028.jpg
2,0 0.044431977 69.078 164.76321 374.48932 380.1...,batch_01_vt/0031.jpg
3,0 0.035923656 0.0 231.00485 17.860897 279.0404...,batch_01_vt/0032.jpg
4,0 0.14912185 0.0 489.8957 19.000397 511.51944 ...,batch_01_vt/0070.jpg


In [8]:
submission

,PredictionString,image_id
0,0 0.030689314 0.0 263.06998 21.266968 303.166 ...,batch_01_vt/0021.jpg
1,0 0.043173574 283.237 232.90508 484.0651 427.2...,batch_01_vt/0028.jpg
2,0 0.044431977 69.078 164.76321 374.48932 380.1...,batch_01_vt/0031.jpg
3,0 0.035923656 0.0 231.00485 17.860897 279.0404...,batch_01_vt/0032.jpg
4,0 0.14912185 0.0 489.8957 19.000397 511.51944 ...,batch_01_vt/0070.jpg
...,...,...
832,0 0.056491163 465.97833 354.56598 497.93417 39...,batch_03/0947.jpg
833,0 0.10238617 492.0802 362.56454 509.76755 393....,batch_03/0968.jpg
834,0 0.058715254 0.0 351.12012 15.586731 377.8006...,batch_03/0969.jpg
835,0 0.05971015 0.0 235.16156 64.856964 413.7744 ...,batch_03/0992.jpg
